In [235]:
# Para llevar el registro
!pip install -q comet_ml
import comet_ml
comet_ml.init(project_name = 'FineTunning')

In [236]:
import pandas as pd
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input, Conv2D, MaxPooling2D, Concatenate, PReLU
from tensorflow.keras.optimizers import RMSprop, SGD, Adam, Adadelta
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.regularizers import l2

In [237]:
# Se crea el experimento en Comet
experiment = comet_ml.Experiment(
    auto_histogram_weight_logging = True,
    auto_histogram_gradient_logging = True,
    auto_histogram_activation_logging = True,
    log_code = True
)
experiment.set_name('Test 26')

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/kaggle/working' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/gurumasterahb/finetunning/c454db0ccb2b4865b456342a7721588f



In [238]:
# Parámetros
i_width = int(178 * 1.2)
i_height = int(218 * 1.2)
input_shape = (i_width, i_height, 3)

parameters = {
    'num_class' : 2,
    'epochs' : 350,
    'batch_size' : 16,
    'learning_rate' : 0.01,
    'loss' : 'binary_crossentropy',
    'optimizer' : 'RMSprop'}

experiment.log_parameters(parameters)

In [239]:
# Loading images
train_dir = '/kaggle/input/data-finetunning/Norma_align/Norma_align/train'
val_dir = '/kaggle/input/data-finetunning/Norma_align/Norma_align/val'

# Image Generator
gen_train = ImageDataGenerator(
    rotation_range = 30,
    width_shift_range = 0.15,  # % de desplazamiento horizontal
    height_shift_range = 0.15, # % de desplazamiento vertical
    brightness_range = [0.25, 1.25], # Porcentaje de brillo en el rango
    zoom_range = 0.3, # Zoom de hasta 30%
    channel_shift_range = 20, # Cada pixel de cada canal varía su intensidad en una vecindad de 20 unidades
    fill_mode = 'nearest', # Después de desplazamientos que dejan la imágen 'incompleta' se rellena con el valor de los pixeles más cercanos
    horizontal_flip = True,
    vertical_flip = True,
    rescale = 1./255.)
train = gen_train.flow_from_directory(
    train_dir,
    target_size = (i_width, i_height),
    color_mode = 'rgb',
    class_mode = 'binary',
    batch_size = parameters['batch_size'],
    shuffle = True)

gen_val = ImageDataGenerator(
    rotation_range = 30,
    width_shift_range = 0.15,
    height_shift_range = 0.15,
    brightness_range = [0.25, 1.25],
    zoom_range = 0.3,
    channel_shift_range = 20,
    fill_mode = 'nearest',
    horizontal_flip = True,
    vertical_flip = True,
    rescale = 1./255.)
val = gen_val.flow_from_directory(
    val_dir,
    target_size = (i_width, i_height),
    color_mode = 'rgb',
    class_mode = 'binary',
    batch_size = parameters['batch_size'],
    shuffle = True)

Found 120 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [240]:
# Loading model
path_load = os.path.join('/kaggle/input/data-finetunning/best_val_loss.h5')
model_loaded = tf.keras.models.load_model(path_load)

model_0 = Sequential()

for layer in model_loaded.layers[:15]:
    layer.trainable = False
    model_0.add(layer)

# model_0.add(Dense(128, activation = 'relu', kernel_regularizer = l2(0.01)))
model_0.add(Dense(512, activation = 'PReLU'))
model_0.add(Dense(256, activation = 'PReLU'))
# model_0.add(Dropout(0.12))
model_0.add(Dense(128, activation = 'relu', kernel_regularizer = l2(0.05)))
model_0.add(Dropout(0.5))
model_0.add(Dense(64, activation = 'relu', kernel_regularizer = l2(0.02)))
model_0.add(Dense(1, activation = 'sigmoid'))
    
# model_0.summary()

In [241]:
# Checkpoint para el mejor modelo (val_accuracy)
filepath = 'best_val_loss.h5'
checkpoint = ModelCheckpoint(
    filepath,
    monitor = 'val_loss',
    verbose = 1,
    save_best_only = True,
    mode = 'min')

In [242]:
# Compilación y entrenamiento
model_0.compile(
    loss = parameters['loss'],
    optimizer = RMSprop(learning_rate = parameters['learning_rate']),
    metrics = ['accuracy'])

model_0.fit(
    train,
    epochs = parameters['epochs'],
    verbose = 1,
    validation_data = val,
    callbacks = [checkpoint])

Epoch 1/350
8/8 [==============================] - ETA: 0s - loss: 3.7264 - accuracy: 0.4333
Epoch 1: val_loss improved from inf to 1.00274, saving model to best_val_loss.h5


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 6s 466ms/step - loss: 3.7264 - accuracy: 0.4333 - val_loss: 1.0027 - val_accuracy: 0.5000
Epoch 2/350
8/8 [==============================] - ETA: 0s - loss: 0.8817 - accuracy: 0.5333
Epoch 2: val_loss did not improve from 1.00274
8/8 [==============================] - 3s 422ms/step - loss: 0.8817 - accuracy: 0.5333 - val_loss: 1.8507 - val_accuracy: 0.5000
Epoch 3/350
8/8 [==============================] - ETA: 0s - loss: 1.0500 - accuracy: 0.4750
Epoch 3: val_loss improved from 1.00274 to 0.75622, saving model to best_val_loss.h5
8/8 [==============================] - 3s 469ms/step - loss: 1.0500 - accuracy: 0.4750 - val_loss: 0.7562 - val_accuracy: 0.5750
Epoch 4/350
8/8 [==============================] - ETA: 0s - loss: 0.7930 - accuracy: 0.6167
Epoch 4: val_loss improved from 0.75622 to 0.73628, saving model to best_val_loss.h5
8/8 [==============================] - 3s 440ms/step - loss: 0.7930 - accuracy: 0.6167 - val_loss: 0.7363 - val_accur

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 3s 463ms/step - loss: 0.6778 - accuracy: 0.7500 - val_loss: 0.5424 - val_accuracy: 0.8000
Epoch 15/350
8/8 [==============================] - ETA: 0s - loss: 0.6163 - accuracy: 0.7750
Epoch 15: val_loss did not improve from 0.54245
8/8 [==============================] - 3s 427ms/step - loss: 0.6163 - accuracy: 0.7750 - val_loss: 0.6879 - val_accuracy: 0.7000
Epoch 16/350
8/8 [==============================] - ETA: 0s - loss: 0.6179 - accuracy: 0.7333
Epoch 16: val_loss did not improve from 0.54245
8/8 [==============================] - 4s 481ms/step - loss: 0.6179 - accuracy: 0.7333 - val_loss: 0.6899 - val_accuracy: 0.7000
Epoch 17/350
8/8 [==============================] - ETA: 0s - loss: 0.7002 - accuracy: 0.7417
Epoch 17: val_loss did not improve from 0.54245
8/8 [==============================] - 4s 486ms/step - loss: 0.7002 - accuracy: 0.7417 - val_loss: 0.6245 - val_accuracy: 0.7500
Epoch 18/350
8/8 [==============================] - ETA: 0

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 3s 424ms/step - loss: 0.5632 - accuracy: 0.7833 - val_loss: 0.5253 - val_accuracy: 0.7500
Epoch 28/350
8/8 [==============================] - ETA: 0s - loss: 0.4870 - accuracy: 0.8417
Epoch 28: val_loss did not improve from 0.52534
8/8 [==============================] - 3s 415ms/step - loss: 0.4870 - accuracy: 0.8417 - val_loss: 0.6445 - val_accuracy: 0.8000
Epoch 29/350
8/8 [==============================] - ETA: 0s - loss: 0.5664 - accuracy: 0.8250
Epoch 29: val_loss did not improve from 0.52534
8/8 [==============================] - 3s 413ms/step - loss: 0.5664 - accuracy: 0.8250 - val_loss: 0.6108 - val_accuracy: 0.7000
Epoch 30/350
8/8 [==============================] - ETA: 0s - loss: 0.7168 - accuracy: 0.6917
Epoch 30: val_loss did not improve from 0.52534
8/8 [==============================] - 3s 435ms/step - loss: 0.7168 - accuracy: 0.6917 - val_loss: 0.7547 - val_accuracy: 0.6250
Epoch 31/350
8/8 [==============================] - ETA: 0

In [243]:
experiment.log_model("Facial_r", filepath)
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/gurumasterahb/finetunning/c454db0ccb2b4865b456342a7721588f
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [350]                : (0.4333333373069763, 0.9083333611488342)
COMET INFO:     batch_accuracy [350]          : (0.125, 1.0)
COMET INFO:     batch_loss [350]              : (0.13561850786209106, 10.973121643066406)
COMET INFO:     epoch_duration [350]          : (2.7555110250032158, 5.821397267001885)
COMET INFO:     loss [350]                    : (0.38706174492836, 3.7263667583465576)
COMET INFO:     val_accuracy [350]            : (0.4749999940395355, 0.8999999761581421)
COMET INFO:     val_loss [350] 